In [1]:
import warnings

# 모든 경고 무시
warnings.filterwarnings("ignore")


In [2]:
#필요 패키지 import

import io
import zipfile
import requests
from xml.etree.ElementTree import parse
import pandas as pd
import time
import datetime
import math
import pickle


In [24]:
#전자공시 API 인증키
# crtfc_key='8aa206443037bf0e34e97d743a1da1391ead1e3c'
crtfc_key='6ac6c88791643c87a1e2e0b33040e9c953a8287d'

In [42]:
# 전자공시 api 회사정보 가져오기
path='/Users/minjunglee/대학원/2023-가을학기/'
filename='corpcode.zip'

url = 'https://opendart.fss.or.kr/api/corpCode.xml'
params = {
    'crtfc_key': crtfc_key,
}
results=requests.get(url, params=params)

file = open(path+filename, 'wb')
file.write(results.content)
file.close()

zipfile.ZipFile(path+filename).extractall(path)

tree = parse(path + 'CORPCODE.xml')
root = tree.getroot()
li=root.findall('list')
corp_code,corp_name,stock_code,modify_date=[],[],[],[]
for d in li:
    corp_code.append(d.find('corp_code').text)
    corp_name.append(d.find('corp_name').text)
    stock_code.append(d.find('stock_code').text)
    modify_date.append(d.find('modify_date').text)
corps_df = pd.DataFrame({'corp_code':corp_code,'corp_name':corp_name,
         'stock_code':stock_code,'modify_date':modify_date})

corps_df = corps_df.loc[corps_df['stock_code']!=' ',:].reset_index(drop=True)

In [47]:
# 회사리스트 별로 전자공시 기업개황 api 호출 후 결과저장_all corps
result_all=[]
corps_detail=pd.DataFrame()

print('총 기업수는 : ' + str(corps_df.shape[0]))

# 수집한 회사에 대해서 for문.
for i, r in corps_df.iterrows():
    #if i == 2:
    #    break

    #없으면 어느 시점에서 에러발생
    time.sleep(0.2)

    # print('i = ' + str(i))
    corp_code=str(r['corp_code'])
    corp_name=r['corp_name']

    url = 'https://opendart.fss.or.kr/api/company.json'
    params = {
        'crtfc_key': crtfc_key,
        'corp_code' : corp_code,
    }

    results = requests.get(url, params=params, verify=False).json()
    # print(results)
    # 응답이 정상 '000' 일 경우에만 데이터 수집
    if results['status'] == '000':
        result_all.append(results)
    else:
        print('error',i)
    
     # 1000개마다 로그 출력
    if i % 1000 == 0:
        print(f'Processed {i} items')

print(i)
corps_detail=pd.DataFrame(result_all)

총 회사수는 : 3648
Processed 0 items
Processed 1000 items
Processed 2000 items
Processed 3000 items
error 3559
error 3560
error 3561
error 3562
error 3563
error 3564
error 3565
error 3566
error 3567
error 3568
error 3569
error 3570
error 3571
error 3572
error 3573
error 3574
error 3575
error 3576
error 3577
error 3578
error 3579
error 3580
error 3581
error 3582
error 3583
error 3584
error 3585
error 3586
error 3587
error 3588
error 3589
error 3590
error 3591
error 3592
error 3593
error 3594
error 3595
error 3596
error 3597
error 3598
error 3599
error 3600
error 3601
error 3602
error 3603
error 3604
error 3605
error 3606
error 3607
error 3608
error 3609
error 3610
error 3611
error 3612
error 3613
error 3614
error 3615
error 3616
error 3617
error 3618
error 3619
error 3620
error 3621
error 3622
error 3623
error 3624
error 3625
error 3626
error 3627
error 3628
error 3629
error 3630
error 3631
error 3632
error 3633
error 3634
error 3635
error 3636
error 3637
error 3638
error 3639
error 3640
err

In [126]:
#전자공시 기업개황 전체 리스트 저장
#1.csv
corps_detail.to_csv('corps_detail_raw_data.csv')

#2.pickle
with open('corps_detail_raw_data.pickle','wb') as fw:
    pickle.dump(corps_detail,fw)

In [5]:
# 전체리스트 불러오기
corps_detail =pd.read_csv('corps_detail_raw_data.csv', index_col=0)

In [6]:
corps_detail

,status,message,corp_code,corp_name,corp_name_eng,stock_name,stock_code,ceo_nm,corp_cls,jurir_no,bizr_no,adres,hm_url,ir_url,phn_no,fax_no,induty_code,est_dt,acc_mt
0,0,정상,260985,한빛네트,"HanbitNet, Inc.",한빛네트,36720,강승환,E,1.101111e+12,2088124283,서울특별시 종로구 익선동 30-6,www.hanbitnet.com,NaN,02-3668-0114,02-3668-0112,58221,19970722,12
1,0,정상,264529,(주)엔플렉스,"Nplex,Inc.",엔플렉스,40130,이 정 학,E,1.101112e+12,2208156354,서울특별시 강남구 논현동 98 동익빌딩 2층,www.gamekorea.net,NaN,(02) 3453-7111,(02) 3453-7345,58221,19981030,12
2,0,정상,358545,(주)동서정보기술,"Dong Seo Information Technology Co., Ltd.",동서정보기술,55000,이희봉,E,1.101111e+12,2208103653,서울특별시 강남구 대치동 997-4,http://www.dsit.co.kr,NaN,02-565-4510 (220),02-3453-4552,58221,19960410,12
3,0,정상,231567,주식회사 애드모바일,NaN,애드모바일,32600,김용석,E,1.101111e+12,1208151156,서울특별시 강서구 가양동 1458-9 동남빌딩 2층,www.kdline.co.kr,NaN,02-518-0101 (변광섭),02-518-0133,58221,19950614,12
4,0,정상,247939,(주)씨모스,"creative master of semiconductor. co.,ltd",씨모스,37600,유인엽,E,1.201110e+12,1398120545,인천광역시 남동구 고잔동 685-6,www.cmos21.co.kr,NaN,032-814-0981-4,032-814-0985,29294,19950509,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3554,0,정상,260408,(주)팜스코,FARMSCO,팜스코,36580,정학상,Y,1.311110e+12,1298130202,경기도 안성시 미양면 제2공단4길 33,www.farmsco.com,NaN,02-6200-9901,02-6200-9908,106,19991001,12
3555,0,정상,175623,(주)ES큐브,"ES CUBE CO., LTD.",ES큐브,50120,안경환,K,1.801110e+12,6218121761,"서울특별시 강남구 강남대로 556 12층(논현동, 논현빌딩)",www.escube.co.kr,www.escube.co.kr,02-3446-4875,02-3446-4873,46419,19770620,12
3556,0,정상,422895,(주)세이브존아이앤씨,SAVEZONE I&C Corp.,세이브존I&C,67830,"김현동, 이상준",Y,1.101113e+12,2108141081,서울특별시 노원구 한글비석로 57 (하계동),www.savezone.co.kr,NaN,032-320-9032,031-930-9020,47111,20020508,12
3557,0,정상,1055317,(주)비엔디생활건강,"B&D Life Health CO.,LTD",비엔디생활건강,215050,이다니엘,E,1.101115e+12,2158763345,충청북도 음성군 생극면 생극산단길 31(주)비엔디생활건강,www.e-seje.com,NaN,02-552-8481,025528482,20421,20120105,12


In [4]:
#esg 데이터 read
corps_list = pd.read_csv('/Users/minjunglee/대학원/2023-가을학기/esg_grade_mac.csv')

In [62]:
corps_list

,년도,회사명,종합등급,환경,사회,지배구조,한국ESG연구소,서스틴베스트,Moody,MSCI,SP
0,2020,AJ네트웍스,B,D,B,B,-,-,-,-,-
1,2020,AK홀딩스,B+,C,B+,B+,-,-,-,-,-
2,2020,BGF,B+,B,A,B+,-,-,-,-,-
3,2020,BGF리테일,A,B+,B+,A,-,-,-,BB,-
4,2020,BNK금융지주,A+,B+,A+,A+,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...
3135,2023,휠라홀딩스,B+,B,A,B+,A,A,24,-,42
3136,2023,휴니드테크놀러지스,D,D,C,C,-,-,-,-,-
3137,2023,휴비스,B+,B+,A,B+,-,-,-,-,6
3138,2023,휴스틸,D,D,D,C,-,-,-,-,-


In [9]:
esg_firm_list =corps_list['회사명'].unique()
len(corps_list['회사명'].unique()) # 전체 793개 기업

793

In [10]:
#전자공시에서 받은 기업개황 중 esg평가 기업만 필터링
esg_corps_detail = corps_detail[corps_detail['stock_name'].isin(esg_firm_list)]

In [ ]:
##시간 재는 코드(참고)
start = time.time()
math.factorial(100000)
end = time.time()

print(f"{end - start:.5f} sec")


In [28]:
import pandas as pd
from time import sleep

# 회사별 전체재무제표 확인
result_all = pd.DataFrame()


# bsns_year=['2023','2022','2021','2020','2019']
bsns_year=['2023']
reprt_code=['11011','11014','11012','11013']#1분기보고서: 11013 반기보고서: 11012 3분기보고서: 11014 사업보고서: 11011
# reprt_code=['11011']#1분기보고서: 11013 반기보고서: 11012 3분기보고서: 11014 사업보고서: 11011
fs_div='OFS'  #CFS:연결재무제표, OFS:재무제표

# corp_detail = pd.DataFrame({'corp_code':['00126380'],'corp_name':['삼성전자'],'stock_code':['095570']})
# corp_detail = pd.DataFrame({'corp_code':['00365387'],'corp_name':['AJ네트웍스'],'stock_code':['095570']})
cnt=0
start = time.time()
for i, r in esg_corps_detail.iterrows():
    #없으면 어느 시점에서 에러발생
    time.sleep(0.2)

    #전체재무제표 요청인자
    crtfc_key=crtfc_key
    corp_code=str(r['corp_code']).zfill(8)

    url = 'https://opendart.fss.or.kr/api/fnlttSinglAcnt.json' #단일회사 재무제표
    # url = 'https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json' # 단일회사 전체 재무제표
    # url = 'https://opendart.fss.or.kr/api/fnlttSinglIndx.json' #단일회사 주요 재무제표
    # url = 'https://opendart.fss.or.kr/api/list.json'
    for x in bsns_year:
        for y in reprt_code:
            params = {
                'crtfc_key': crtfc_key,
                'corp_code' : corp_code,
                'bsns_year' : x,
                'reprt_code' : y,
                'fs_div' : fs_div,
            }

            #결과를 json형태로 저장
            results = requests.get(url, params=params,verify=False).json()
            # print(results)
            if results['status'] == '000':
                result_df = pd.DataFrame(results['list'])
                result_all = pd.concat([result_all,result_df])
            # 1000개마다 로그 출력
            if cnt % 100 == 0:
                print(f'Processed {cnt} items')
                end = time.time()
                print(f"{end - start:.5f} sec")
                start = time.time()
                cnt=+1

# 끝나고 마지막 로그 출력
print('Finished processing all items')

end = time.time()
print(f"{end - start:.5f} sec")
result_all

Processed 0 items
0.31375 sec


SSLError: HTTPSConnectionPool(host='opendart.fss.or.kr', port=443): Max retries exceeded with url: /api/list.json?crtfc_key=d04c4e316495dc1fe0628dd8a022490682676c56&corp_code=00369833&bgn_de=20230101&end_de=20231030&pblntf_ty=C (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1129)')))

In [26]:
# 전자공시 공시정보 저장
# result_all.to_csv('fnlttSingAcnt_2023_raw_data.csv', index=False) #37288건
# result_all.to_csv('fnlttSingAcnt_2022_raw_data.csv', index=False) # 74000건
# result_all.to_csv('fnlttSingAcnt_2021_raw_data.csv', index=False) #73889건
# result_all.to_csv('fnlttSingAcnt_2020_raw_data.csv', index=False) #72687건
# result_all.to_csv('fnlttSingAcnt_2019_raw_data.csv', index=False) #71511건 
# result_all.to_csv('fnlttSinglAcntAll_2019_raw_data.csv', index=False) #390078건 
# result_all.to_csv('fnlttSinglAcntAll_2020_raw_data.csv', index=False) #390869건 
# result_all.to_csv('fnlttSinglAcntAll_2022_raw_data.csv', index=False) #392020건 
result_all.to_csv('fnlttSinglAcntAll_2023_raw_data.csv', index=False) #185167건 

In [ ]:
result_all.columns

Index([], dtype='object')

In [ ]:
# 연도별 데이터 불러와서 하나로 합치기


In [ ]:
#필요한 컬럼만 추출

In [ ]:
# filter_list =['유동자산', '현금및현금성자산', '단기금융상품', '매출채권', '미수금', '선급비용', '재고자산', '기타유동자산', '비유동자산', '기타포괄손익-공정가치금융자산',
#               '당기손익-공정가치금융자산', '종속기업, 관계기업 및 공동기업 투자', '유형자산', '무형자산', '순확정급여자산', '이연법인세자산', '기타비유동자산', '자산총계', '유동부채', '매입채무', '단기차입금']

filter_list =['유동자산', '비유동자산', '자산총계', '유동부채', '비유동부채', '부채총계', '자본금', '이익잉여금',
       '자본총계', '매출액', '영업이익', '법인세차감전 순이익', '당기순이익', '당기순이익(손실)']

# columns=['rcept_no', 'reprt_code', 'bsns_year', 'corp_code', 'stock_code',
#        'fs_div', 'fs_nm', 'sj_div', 'sj_nm', 'account_nm', 'thstrm_nm',
#        'thstrm_dt', 'thstrm_amount', 'frmtrm_nm', 'frmtrm_dt', 'frmtrm_amount',
#        'ord', 'currency', 'thstrm_add_amount', 'frmtrm_add_amount']

In [ ]:
import re
# filter_list=['유동자산']
# 필터링할 단어들을 정규표현식 패턴으로 변환
pattern = '|'.join(filter_list)
pattern = r'\b(?:{})\b'.format(pattern)

# account_nm 열에서 패턴과 일치하는 값을 가지는 행 선택
filtered_df = result_all[result_all['account_nm'].str.contains(pattern, regex=True)]

# filtered_df=result_all[result_all['account_nm'].apply(lambda x: all(item in x for item in filter_list))]

- 유동자산: 1년 이내에 현금화되거나 소비되는 자산으로, 단기금융상품, 매출채권, 재고자산 등이 포함됩니다.
-현금및현금성자산: 현금과 현금과 유사한 형태의 자산을 의미합니다. 보통은 현금 잔고와 예금 등이 포함됩니다.
-단기금융상품: 1년 이내에 현금화될 수 있는 금융상품으로, 예금증서, 상업어음, 단기채권 등이 포함됩니다.
-매출채권: 기업이 제품이나 서비스를 판매한 후 고객으로부터 받는 지급요구권입니다.
-미수금: 기업이 판매한 제품이나 서비스에 대한 대금을 아직 받지 못한 상태의 자산입니다.
-선급비용: 기업이 미래에 활동에 지출할 비용을 미리 지불한 상태의 자산입니다.
-재고자산: 기업이 생산한 제품이나 구매한 상품의 재고로, 판매를 위해 보유하는 자산입니다.
-기타유동자산: 유동자산에 속하지만 다른 유형에 해당하지 않는 자산입니다.
-비유동자산: 장기적으로 현금화되는 자산으로, 유형자산, 무형자산 등이 포함됩니다.
-기타포괄손익-공정가치금융자산: 기업이 보유한 금융자산의 가치가 변동함에 따라 발생하는 손익입니다.
-당기손익-공정가치금융자산: 기업이 보유한 금융자산의 가치 변동에 따라 직접적으로 발생하는 손익입니다.
-종속기업, 관계기업 및 공동기업 투자: 기업이 다른 기업에 투자한 자산입니다.
-유형자산: 기업이 소유하고 운영하는 부동산, 설비, 기계 등의 자산입니다.
-무형자산: 기업의 재산권이나 권리로 표현되는 자산으로, 특허, 상표, 소프트웨어 등이 포함됩니다.
-순확정급여자산: 기업이 앞으로 지급해야 할 확정급여의 현재 가치입니다.
-이연법인세자산: 기업이 현재 기간에 납부하지 않은 법인세의 차출금입니다.
-기타비유동자산: 비유동자산에 속하지만 다른 유형에 해당하지 않는 자산입니다.
-자산총계: 기업이 보유하고 있는 모든 자산의 총액입니다.
-유동부채: 1년 이내에 상환해야 할 부채로, 단기차입금, 매입채무, 미지급금 등이 포함됩니다.
-매입채무: 기업이 구매한 상품이나 서비스에 대한 대금을 아직 지불하지 않은 상태의 부채입니다.
-단기차입금: 기업이 1년 이내에 상환해야 할 차입금을 의미합니다.

In [ ]:
# 파일 저장
filtered_df.to_csv('fnlttAcnt_raw_data.csv', index=False)

In [ ]:
pivot_df=filtered_df.pivot(index=['corp_code', 'bsns_year', 'reprt_code','fs_div'], columns='account_nm', values='frmtrm_amount').reset_index()

In [ ]:
pivot_df

account_nm,corp_code,bsns_year,reprt_code,fs_div,당기순이익,당기순이익(손실),매출액,법인세차감전 순이익,부채총계,비유동부채,비유동자산,영업이익,유동부채,유동자산,이익잉여금,자본금,자본총계,자산총계
0,00365387,2019,11011,CFS,"37,268,842,438","37,268,842,438","1,045,526,121,935","-49,408,471,511","2,083,331,488,612","474,261,316,507","1,063,068,630,148","-21,236,052,082","1,609,070,172,105","1,512,070,831,211","157,174,672,951","46,822,295,000","491,807,972,747","2,575,139,461,359"
1,00365387,2019,11011,OFS,"-22,339,664,361","-22,339,664,361","339,799,797,610","-17,009,322,453","809,377,040,749","367,289,813,891","816,623,294,301","37,338,422,646","442,087,226,858","258,953,988,873","142,392,992,995","46,822,295,000","266,200,242,425","1,075,577,283,174"
2,00365387,2019,11012,CFS,"-4,738,628,954","-4,738,628,954","254,464,921,024","-5,985,734,500","2,083,331,488,612","474,261,316,507","1,063,783,463,551","1,702,931,102","1,609,070,172,105","1,511,355,997,808","157,174,672,951","46,822,295,000","491,807,972,747","2,575,139,461,359"
3,00365387,2019,11012,OFS,"-2,360,890,899","-2,360,890,899","83,660,157,180","-917,310,245","809,377,040,749","367,289,813,891","817,338,127,704","9,475,880,052","442,087,226,858","258,239,155,470","142,392,992,995","46,822,295,000","266,200,242,425","1,075,577,283,174"
4,00365387,2019,11013,CFS,"9,982,225,625","9,982,225,625","258,787,776,363","5,645,908,724","2,066,874,619,886","474,261,316,507","1,063,783,463,551","4,250,327,841","1,592,613,303,379","1,511,355,997,808","208,668,379,764","46,822,295,000","508,264,841,473","2,575,139,461,359"
5,00365387,2019,11013,OFS,"4,103,821,090","4,103,821,090","88,862,996,589","5,791,038,986","809,377,040,749","367,289,813,891","817,338,127,704","9,768,934,693","442,087,226,858","258,239,155,470","142,392,992,995","46,822,295,000","266,200,242,425","1,075,577,283,174"
6,00365387,2019,11014,CFS,"4,897,453,512","4,897,453,512","269,069,659,805","1,456,149,987","2,083,331,488,612","474,261,316,507","1,063,783,463,551","9,299,628,964","1,609,070,172,105","1,511,355,997,808","157,174,672,951","46,822,295,000","491,807,972,747","2,575,139,461,359"
7,00365387,2019,11014,OFS,"3,490,261,843","3,490,261,843","82,720,210,428","5,366,397,879","809,377,040,749","367,289,813,891","817,338,127,704","10,216,012,909","442,087,226,858","258,239,155,470","142,392,992,995","46,822,295,000","266,200,242,425","1,075,577,283,174"
8,00365387,2020,11011,CFS,"42,128,594,358","42,128,594,358","1,000,259,401,598","107,048,170,172","1,455,914,070,366","659,076,988,337","1,302,199,830,881","15,649,322,684","796,062,818,789","478,799,539,688","198,771,796,598","46,822,295,000","347,341,155,620","1,803,255,225,986"
9,00365387,2020,11011,OFS,"50,738,602,999","50,738,602,999","391,780,284,062","87,201,170,590","921,116,846,324","405,811,872,080","957,681,241,619","39,951,143,707","515,304,974,244","273,884,243,596","188,449,366,494","46,822,295,000","310,448,638,891","1,231,565,485,215"


In [ ]:
final_df = pivot_df.reset_index()

In [ ]:
final_df

account_nm,corp_code,bsns_year,reprt_code,기말 현금및현금성자산,기초 현금및현금성자산,기타비유동자산,기타유동자산,단기차입금,당기말 현금및현금성자산,매입채무,...,반기말 현금및현금성자산,분기말 현금및현금성자산,비유동자산,유동부채,유동자산,유형자산,이연법인세자산,자산총계,재고자산,현금및현금성자산
0,00365387,2021,11011,NaN,13201292466,2989790083,26144186315,42500000000,15573060373,31243339043,...,NaN,NaN,1067752052896,397997623454,130457258750,57692877872,855316948,1198209311646,8754240693,15573060373
1,00365387,2021,11012,48374783453,13201292466,4531284300,5638559449,82500000000,NaN,26913192029,...,NaN,NaN,1015907828167,428820150438,148202170566,57160440740,5996194457,1164109998733,18949153138,48374783453
2,00365387,2021,11013,35280507535,13201292466,1414618363,13408444589,150500000000,NaN,19339989686,...,NaN,NaN,994592943625,515468726558,166242235197,57526787437,16329038763,1218314937498,19763373050,35280507535
3,00365387,2021,11014,14861538166,13201292466,3347821720,6296194475,78000000000,NaN,32566071248,...,NaN,NaN,1090393348957,429205945257,124696034513,56915478088,4183431827,1215089383470,18485358547,14861538166
4,00365387,2022,11011,NaN,15573060373,3375320816,10950547607,193971100000,99647782619,42857596634,...,NaN,NaN,1134430052896,775956063610,237632604285,61017187499,12604832277,1372062657181,19999591332,99647782619
5,00365387,2022,11012,NaN,15573060373,3811036333,19966107067,109500000000,NaN,34613767694,...,31118050941,NaN,1079644910419,510170857556,182209224997,58432271742,677866906,1261854135416,26573450358,31118050941
6,00365387,2022,11013,NaN,15573060373,2794642643,15274303794,105500000000,NaN,33647712198,...,NaN,34542606676,1059050727492,493377808213,157283046094,58406134600,694892224,1216333773586,22595675983,34542606676
7,00365387,2022,11014,NaN,15573060373,4275750854,17535841247,190300000000,NaN,37680230147,...,NaN,46720939335,1138001704938,654957144422,218541779271,59687906526,669149687,1356543484209,29440069529,46720939335
8,00365387,2023,11012,NaN,99647782619,1656003291,7106796015,113938400000,NaN,44827903503,...,81340210229,NaN,1205303662718,637331693953,214425309466,104321916447,5656830694,1419728972184,23488464666,81340210229
9,00365387,2023,11013,NaN,99647782619,3211681115,10748606942,168126600000,NaN,40826077952,...,NaN,83164677262,1186348984352,733306490740,203729943614,101673281914,5656830694,1390078927966,21318748132,83164677262


In [ ]:
# 파일 저장
final_df.to_csv('fnlttAcntAll_raw_data.csv', index=False)

In [ ]:
import pickle

# pickle 파일 경로
file_path = "/content/drive/MyDrive/Colab Notebooks/대학원/캡스톤/1024/fnlttAcntAll _raw_data.pickle"

# pickle 파일 읽기
with open(file_path, "rb") as f:
    data = pickle.load(f)

In [ ]:
data

,rcept_no,reprt_code,bsns_year,corp_code,sj_div,sj_nm,account_id,account_nm,account_detail,thstrm_nm,thstrm_amount,frmtrm_nm,frmtrm_amount,bfefrmtrm_nm,bfefrmtrm_amount,ord,currency,thstrm_add_amount
0,20230915000256,11011,2023,00138747,BS,재무상태표,ifrs-full_CurrentAssets,유동자산,-,제 45 기,114986815860,제 44 기,87669584025,제 43 기,80452873185,1,KRW,NaN
1,20230915000256,11011,2023,00138747,BS,재무상태표,ifrs-full_CashAndCashEquivalents,현금및현금성자산,-,제 45 기,6795734003,제 44 기,11696458267,제 43 기,12375818651,2,KRW,NaN
2,20230915000256,11011,2023,00138747,BS,재무상태표,dart_ShortTermDepositsNotClassifiedAsCashEquiv...,단기금융상품,-,제 45 기,399999985,제 44 기,3240120000,제 43 기,,3,KRW,NaN
3,20230915000256,11011,2023,00138747,BS,재무상태표,ifrs-full_TradeAndOtherCurrentReceivables,매출채권 및 기타유동채권,-,제 45 기,46346281516,제 44 기,41674393205,제 43 기,39875166773,4,KRW,NaN
4,20230915000256,11011,2023,00138747,BS,재무상태표,ifrs-full_Inventories,재고자산,-,제 45 기,59508676713,제 44 기,29808239467,제 43 기,26447090836,5,KRW,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,20230823000023,11011,2023,01437292,SCE,자본변동표,ifrs-full_DividendsPaid,배당금지급,별도재무제표 [member],제 6 기,-2713500000,제 5 기,-3015000000,제 4 기,-243210000,4,KRW,NaN
61,20230823000023,11011,2023,01437292,SCE,자본변동표,ifrs-full_Equity,기말자본,자본 [member]|자본잉여금 [member],제 6 기,74207866689,제 5 기,74207866689,제 4 기,74218485078,5,KRW,NaN
62,20230823000023,11011,2023,01437292,SCE,자본변동표,ifrs-full_Equity,기말자본,자본 [member]|이익잉여금 [member],제 6 기,-4432669487,제 5 기,4432669487,제 4 기,2061573423,5,KRW,NaN
63,20230823000023,11011,2023,01437292,SCE,자본변동표,ifrs-full_Equity,기말자본,별도재무제표 [member],제 6 기,88584101168,제 5 기,89875197202,제 4 기,92256911655,5,KRW,NaN
